In [ ]:
pip install pyspark

  Using cached pyspark-3.2.1.tar.gz (281.4 MB)
  Using cached py4j-0.10.9.3-py2.py3-none-any.whl (198 kB)
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=43201d97f286e4d675d0ca57d8eee790df8ed9afd67c3798041e8b7d08277d00
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
from pyspark import SparkConf,SparkContext

In [ ]:
if __name__=="__main__":
  conf=SparkConf().setAppName('reviews').setMaster('local')
  sc=SparkContext(conf=conf)

In [ ]:
mydata=sc.textFile("reviews.csv")

In [ ]:
word=mydata.flatMap(lambda x:x.split(" ")) #created a single list for all the words in the file 

In [ ]:
word.take(10)

['"nice',
 'hotel',
 'expensive',
 'parking',
 'got',
 'good',
 'deal',
 'stay',
 'hotel',
 'anniversary,']

In [ ]:
word.countByValue() #count of each every word in the file 

In [ ]:
import re

In [ ]:
#using regular expressions before flatmap and count BY value for better result 

In [ ]:
#getting the required data out of total 
#in our their are some numerical--like -5,"20th"etc
#so using replace we replace all of them with blank
#
newwords=mydata.map(lambda x:x.replace(r"^[a-zA-Z]\x+",""))

In [ ]:
newrd=newwords.flatMap(lambda x:x.split(" "))

In [ ]:
newrd.countByValue()

SET functions in RDD


In [ ]:
rdd1=sc.parallelize([10,20,30,40])
rdd2=sc.parallelize([20,40,60,80,100])

In [ ]:
unionres=rdd1.union(rdd2)
unionres.collect()

[10, 20, 30, 40, 20, 40, 60, 80, 100]

In [ ]:
intersec=rdd1.intersection(rdd2)
intersec.collect()

[20, 40]

In [ ]:
carte=rdd1.cartesian(rdd2)
carte.collect()

[(10, 20),
 (10, 40),
 (10, 60),
 (10, 80),
 (10, 100),
 (20, 20),
 (20, 40),
 (20, 60),
 (20, 80),
 (20, 100),
 (30, 20),
 (30, 40),
 (30, 60),
 (30, 80),
 (30, 100),
 (40, 20),
 (40, 40),
 (40, 60),
 (40, 80),
 (40, 100)]

In [ ]:
sub=rdd1.subtract(rdd2)
sub.collect()

[10, 30]

In [ ]:
sub2=rdd2.subtract(rdd1)
sub2.collect()

[60, 80, 100]

working on raw data

In [ ]:
!pip install wget

In [ ]:
import wget 

In [ ]:
wget.download("https://raw.githubusercontent.com/jleetutorial/python-spark-tutorial/master/in/nasa_19950701.tsv","first.csv")

'first.csv'

In [ ]:
wget.download("https://raw.githubusercontent.com/jleetutorial/python-spark-tutorial/master/in/nasa_19950801.tsv","second.csv")

'second.csv'

In [ ]:
firstrdd=sc.textFile("first.csv")
firstrdd.take(4)   # here separator is \t(tab)

['host\tlogname\ttime\tmethod\turl\tresponse\tbytes',
 '199.72.81.55\t-\t804571201\tGET\t/history/apollo/\t200\t6245\t\t',
 'unicomp6.unicomp.net\t-\t804571206\tGET\t/shuttle/countdown/\t200\t3985\t\t',
 '199.120.110.21\t-\t804571209\tGET\t/shuttle/missions/sts-73/mission-sts-73.html\t200\t4085\t\t']

In [ ]:
firstrdd.count()

10000

In [ ]:
secondrdd=sc.textFile("second.csv")
secondrdd.take(4)

['host\tlogname\ttime\tmethod\turl\tresponse\tbytes',
 'in24.inetnebr.com\t-\t807249601\tGET\t/shuttle/missions/sts-68/news/sts-68-mcc-05.txt\t200\t1839\t\t',
 'uplherc.upl.com\t-\t807249607\tGET\t/\t304\t0\t\t',
 'uplherc.upl.com\t-\t807249608\tGET\t/images/ksclogo-medium.gif\t304\t0\t\t']

In [ ]:
secondrdd.count()

10000

Understanding Intersection,union ,subtract in host data

In [ ]:
# removing header and extracting the required data
secondheader=secondrdd.first()
secondnew=secondrdd.filter(lambda x:x!=secondheader)
secondhost=secondnew.map(lambda x:x.split("\t")[0])
secondhost.count()

9999

In [ ]:
secondhost.take(4)

['in24.inetnebr.com', 'uplherc.upl.com', 'uplherc.upl.com', 'uplherc.upl.com']

In [ ]:
#removing header and extracting the required data 
firstheader=firstrdd.first()
firstnew=firstrdd.filter(lambda x:x!=firstheader)
firsthost=firstnew.map(lambda x:x.split("\t")[0])
firsthost.count()
firsthost.take(4)

['199.72.81.55',
 'unicomp6.unicomp.net',
 '199.120.110.21',
 'burger.letters.com']

In [ ]:
firsthost.union(secondhost).count()

19998

In [ ]:
firsthost.intersection(secondhost).count() #

37

In [ ]:
first=firsthost.subtract(secondhost).count()

In [ ]:
second1=secondhost.subtract(firsthost).count()

In [ ]:
second=secondhost.subtract(firsthost)

In [ ]:
diff=first-second1
print(diff)

-172


In [ ]:
second.coalesce(2).saveAsTextFile("result.csv")